In [32]:
import pandas as pd
import folium
import os

In [44]:
date_folder = '../data/raw/'
#date = '20250518'
date = '20131018'

routes_path = os.path.join(date_folder, date, 'routes.txt')
shapes_path = os.path.join(date_folder, date, 'shapes.txt')
trips_path = os.path.join(date_folder, date, 'trips.txt')
stops_path = os.path.join(date_folder, date, 'stops.txt')

In [45]:
routes = pd.read_csv(routes_path)
shapes = pd.read_csv(shapes_path)
trips = pd.read_csv(trips_path)
stops = pd.read_csv(stops_path)

In [54]:
shape_counts = trips.groupby(['route_id', 'shape_id', 'trip_headsign']).size().reset_index(name='shape_id_count')
shape_counts['shape_rank'] = shape_counts.groupby('route_id')['shape_id_count'].rank(method='first', ascending=False)
most_common_shapes = shape_counts[shape_counts['shape_rank'] <= 2]
#filtered_routes = routes.loc[
#    routes.groupby("route_short_name")["route_sort_order"].idxmin()].reset_index(drop=True).drop(columns=['route_long_name'])

# Ha van route_sort_order, akkor aszerint keressük a legkisebb indexet,
# különben a route_id vagy az eredeti index alapján választunk (itt: első előfordulás).
if "route_sort_order" in routes.columns:
    idx = routes.groupby("route_short_name")["route_sort_order"].idxmin()
else:
    # pl. választhatjuk a minimális indexet – ez megadja az első sort minden csoportból
    idx = routes.groupby("route_short_name").head(1).index

filtered_routes = (
    routes.loc[idx]
          .reset_index(drop=True)
          .drop(columns=["route_long_name"])
)


routes_enriched = pd.merge(filtered_routes, most_common_shapes, on='route_id', how='inner')

In [63]:
routes_enriched[routes_enriched.route_short_name == '19']

,route_id,agency_id,route_short_name,route_desc,route_type,route_color,route_text_color,shape_id,trip_headsign,shape_id_count,shape_rank
442,3190,BKK,19,Batthyány tér M+H / Kelenföld vasútállomás,0,FFEE00,000000,J899,Kelenföld vasútállomás,368,1.0
443,3190,BKK,19,Batthyány tér M+H / Kelenföld vasútállomás,0,FFEE00,000000,J900,Batthyány tér M+H,363,2.0


In [56]:
def get_route_data(route_short_name):
    route_short_name = str(route_short_name)
    shape_ids = routes_enriched[routes_enriched.route_short_name == route_short_name].shape_id.values
    route_shapes = shapes[shapes.shape_id.isin(shape_ids)]
    return route_shapes

In [100]:
route_shapes = get_route_data('19')
route_shapes.sort_values(['shape_id', 'shape_dist_traveled']).head(40)

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled,shape_bkk_ref
39570,J899,47.505895,19.039348,100002,0,31900
39571,J899,47.505481,19.039402,100003,43,31900
39572,J899,47.505319,19.039402,100004,61,31900
39573,J899,47.504699,19.039481,100005,130,31900
39574,J899,47.503853,19.039574,100006,224,31900
39575,J899,47.503089,19.039721,100007,310,31900
39576,J899,47.502279,19.039840,100008,400,31900
39586,J899,47.502234,19.039867,200003,405,31900
39587,J899,47.501488,19.040000,200004,489,31900
39588,J899,47.501218,19.040079,200005,520,31900


In [96]:
def vizualize_route(route_short_name):
    shape_points = get_route_data(route_short_name)
    shape_points = shape_points.sort_values(['shape_id', 'shape_dist_traveled'])#[:24]#[24:100]
    #print(shape_points)

    # térkép középpontja (első pont)
    #center = [shape_points['shape_pt_lat'].iloc[0], shape_points['shape_pt_lon'].iloc[0]]
    center = [shape_points['shape_pt_lat'].mean(), shape_points['shape_pt_lon'].mean()]

    m = folium.Map(location=center, zoom_start=11)

    # vonal kirajzolása
    coords = list(zip(shape_points['shape_pt_lat'], shape_points['shape_pt_lon']))
    folium.PolyLine(coords, color="blue", weight=4.5, opacity=0.8).add_to(m)

    return m  # notebookban megjelenik

In [104]:
vizualize_route('19')